# Generator
- lazy iterator를 리턴하는 함수
- 리스트 등과 달리 lazy iterator은 내용물을 메모리에 저장하지 않는다.
- [참조링크](https://realpython.com/introduction-to-python-generators/)

In [3]:
# Reading Large Files
from csv import reader

csv_gen = reader("data.csv")
row_count = 0

for row in csv_gen:
    row_count += 1

print(f"row count is {row_count}")

# 파일이 매우 크다면?
def csv_reader(file_name):
    for row in open(file_name, "r"):
        yield row

# list comprehension 같은 방법
csv_gen = (row for row in open(file_name))

row count is 8


In [4]:
# generating an infinit sequence
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1

In [7]:
gen = infinite_sequence()
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
1
2
3


In [8]:
def is_palindrome(num):
    if num < 10:
        return False
    temp = num
    reversed_num = 0
    
    while temp != 0:
        reversed_num *= 10 + (temp % 10)
        temp = temp // 10
    
    if num == reversed_num:
        return num
    return False

In [9]:
# for i in infinite_sequence():
#     result = is_palindrome(i)
#     if result:
#         print(result)

In [11]:
# profiling

import sys

nums_squared_lc = [i ** 2 for i in range(10000)]
print(sys.getsizeof(nums_squared_lc))
nums_squared_gc = (i ** 2 for i in range(10000))
print(sys.getsizeof(nums_squared_gc))

87632
128


- If the list is smaller than the running machine’s available memory, then list comprehensions can be faster to evaluate than the equivalent generator expression.

In [13]:
import cProfile
cProfile.run('sum(nums_squared_lc)')

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [14]:
cProfile.run('sum(nums_squared_gc)')

         10005 function calls in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.004    0.000    0.004    0.000 <ipython-input-11-256cbfb399f8>:7(<genexpr>)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.001    0.001    0.005    0.005 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




만약 제너레이터 내의 메서드를 호출할 경우 해당 함수의 코드들은 yield까지 시행된다. 
이를 통해서 제너레이터의 메서드를 부를 때마다 함수 실행을 재개할 수 있다.

In [1]:
def multi_yield():
    yield_str = "this will print the first string"
    yield yield_str
    yield_str = "This will print the second string"
    yield yield_str

In [3]:
multi_obj = multi_yield()
print(next(multi_obj))
print(next(multi_obj))
print(next(multi_obj))

this will print the first string
This will print the second string


StopIteration: 

# Using Advanced Generator Methods

In addition to yield, generator objects can make use of the following methods:
- .send()
- .throw()
- .close()

In [2]:
def is_palindrome(num):
    if num // 10 == 0:
        return False
    temp = num
    reversed_num = 0
    
    while temp != 0:
        reversed_num *= 10 + (temp % 10)
        temp = temp // 10
    
    if num == reversed_num:
        return True
    return False

In [3]:
def infinite_palindrome():
    num = 0
    while True:
        if is_palindrome(num):
            i = (yield num)
            if i is not None:
                num = i
        num += 1

In [ ]:
pal_gen = infinite_palindrome()
for i in pal_gen:
    digits = len(str(i))
    if digits == 5:
        pal_gen.close()
#         pal_gen(10**(digits)).throw(ValueError("we don't like large palindromes"))
    pal_gen.send(10 ** (digits))